**TP**

1) Take any document of your choise and preprocess it using Spacy (learnt in the previous TP and continue the process) (optional: Learn about Gensim Library for a comprehensive understanding)

2. How will will measure the correctness of the output generated? Code some measures and comapare the performances.

3. Read this paper> https://arxiv.org/pdf/2003.07278 . Also, in google scholar search for : 'word embedding survey' or any related keywords. Understand what are the different types.

ok for this task was for  data retereivel  and by comparing the samantic finding semilieriets between different models.

4. You are now supposed to take any one of the type. For example if you take the 'Transformer' architecture and choose the 'BERT'. Now search for all the different varients of BERT and code them separately and compare them.

# **Project**





In [ ]:
# Install libraries into enviroment
!pip install spacy PyPDF2 gensim scikit-learn -q
!python -m spacy download en_core_web_sm -q  # download English model

# Import the libraries
import spacy                         # For natural language processing
from PyPDF2 import PdfReader          # To read text from a PDF file
from gensim.models import Word2Vec, FastText  # For creating word embeddings
from sklearn.metrics.pairwise import cosine_similarity  # To compare sentence similarity
import numpy as np                    # For working with numerical vectors
from collections import Counter       # For counting most common words
import re                             # For text cleaning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 91.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

# STEP 2: LOAD AND READ THE PDF FILE

pdf_path = "paper.pdf"         # Path to your research paper
reader = PdfReader(pdf_path)   # Open the PDF

text = ""                      # Create an empty string to store text

# Loop through all pages in the PDF
for page in reader.pages:
    text += page.extract_text()  # Extract text from each page

print("PDF loaded successfully!")
print(text[:500])                # Print first 500 characters to preview


PDF loaded successfully!
Journal of Artiﬁcial Intelligence Research 65 (2019) 569-631 Submitted 10/2017; published 08/2019
A Survey of Cross-lingual Word Embedding Models
Sebastian Ruder sebastian@ruder.io
Insight Research Centre, National University of Ireland,
Galway, Ireland
Aylien Ltd., Dublin, Ireland
Ivan Vulić iv250@cam.ac.uk
Language Technology Lab, University of Cambridge, UK
Anders Søgaard soegaard@di.ku.dk
University of Copenhagen, Copenhagen, Denmark
Abstract
Cross-lingual representations of words enable us 


In [ ]:
# Remove extra spaces, line breaks, or tabs using REGULAR EXPRESSIONS
text = re.sub(r"\s+", " ", text.strip())

# Print first few lines to check if cleaning worked
print("Cleaned text preview:")
print(text[:300])


Cleaned text preview:
Journal of Artiﬁcial Intelligence Research 65 (2019) 569-631 Submitted 10/2017; published 08/2019 A Survey of Cross-lingual Word Embedding Models Sebastian Ruder sebastian@ruder.io Insight Research Centre, National University of Ireland, Galway, Ireland Aylien Ltd., Dublin, Ireland Ivan Vulić iv250@


In [ ]:
# Now let's process TEXT WITH SPACY

nlp = spacy.load("en_core_web_sm")  # Load English spaCy model
doc = nlp(text)                     # Process the text (tokenization, tagging, entities)

# Print total tokens found
print(f" spaCy processing done! Total tokens: {len(doc)}")

# Show first few sentences
print("\nExample sentences from the PDF:")
for i, sent in enumerate(list(doc.sents)[:3]):
    print(f"{i+1}. {sent.text}")


 spaCy processing done! Total tokens: 35889

Example sentences from the PDF:
1. Journal of Artiﬁcial Intelligence Research 65 (2019) 569-631 Submitted 10/2017; published 08/2019 A Survey of Cross-lingual Word Embedding Models Sebastian Ruder sebastian@ruder.io Insight Research Centre, National University of Ireland, Galway, Ireland Aylien Ltd., Dublin, Ireland Ivan Vulić iv250@cam.ac.uk Language Technology Lab, University of Cambridge, UK Anders Søgaard soegaard@di.ku.dk University of Copenhagen, Copenhagen, Denmark Abstract Cross-lingual representations of words enable us to reason about word meaning in multilingual contexts and are a key facilitator of cross-lingual transfer when developing natural language processing models for low-resource languages.
2. In this survey, we provide a comprehensive typology of cross-lingual word embedding models.
3. We compare their data requirements and objective functions.


This part shows how spaCy breaks down the text into meaningful parts.
For each word, it gives the base form and the part of speech.
It also detects named entities like people, companies, and countries.
This helps us understand the text structure before building embeddings.


In [ ]:
# Now let's break down and show basic Lingustic info

print("\n Example Tokens (Word, Lemma, POS):")
for token in doc[:15]:  # Show only first 15 tokens
    print(f"{token.text:15} | Lemma: {token.lemma_:15} | POS: {token.pos_}")

print("\n Named Entities Found (like names, places, orgs):")
for ent in doc.ents[:10]:
    print(f"{ent.text:30} → {ent.label_}")



 Example Tokens (Word, Lemma, POS):
Journal         | Lemma: Journal         | POS: PROPN
of              | Lemma: of              | POS: ADP
Artiﬁcial       | Lemma: Artiﬁcial       | POS: PROPN
Intelligence    | Lemma: Intelligence    | POS: PROPN
Research        | Lemma: Research        | POS: PROPN
65              | Lemma: 65              | POS: NUM
(               | Lemma: (               | POS: PUNCT
2019            | Lemma: 2019            | POS: NUM
)               | Lemma: )               | POS: PUNCT
569             | Lemma: 569             | POS: NUM
-               | Lemma: -               | POS: PUNCT
631             | Lemma: 631             | POS: NUM
Submitted       | Lemma: submit          | POS: VERB
10/2017         | Lemma: 10/2017         | POS: NUM
;               | Lemma: ;               | POS: PUNCT

 Named Entities Found (like names, places, orgs):
Journal of Artiﬁcial Intelligence Research → ORG
65 (2019                       → DATE
569                         

In [ ]:
# Now we will prepare clean tokens for training

# Keep only meaningful words (nouns, verbs, adjectives, etc.)
allowed_pos = {"NOUN", "PROPN", "VERB", "ADJ", "ADV"}

# Filter and clean tokens
sentences = [
    [t.lemma_.lower() for t in sent if t.is_alpha and not t.is_stop and t.pos_ in allowed_pos]
    for sent in doc.sents
]

# Remove very short sentences (less than 3 words)
sentences = [s for s in sentences if len(s) >= 3]

print(f" Prepared {len(sentences)} sentences for training.")


 Prepared 1356 sentences for training.


Here I trained two embedding models like Word2Vec and FastText on my paper’s text. Word2Vec learns word meanings based on nearby words.
FastText improves that by breaking words into smaller parts (subwords),
so it handles rare or new words better. Each word becomes a 100-dimensional vector that captures its meaning in context.

In [ ]:

# Now I’m training the embedding models using my cleaned text.


# Train Word2Vec model
w2v = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2, sg=1, epochs=20, seed=42)

# Train FastText model (similar to Word2Vec but uses subword(means also learning>learn>ing) info)
ft = FastText(sentences=sentences, vector_size=100, window=5, min_count=2, epochs=20, seed=42)

print(" Both Word2Vec and FastText models trained successfully!")


 Both Word2Vec and FastText models trained successfully!


In this step, I took two sentences with similar meaning and used both models to calculate their similarity. I cleaned and lemmatized the sentences using spaCy, converted them into vectors using Word2Vec and FastText, and then used cosine similarity to measure how close their meanings are.

The higher the score (closer to 1), the more similar the sentences are.
FastText usually performs better because it understands subword patterns and captures richer context.”

In [ ]:
# Two example sentences to test similarity.

# Two sample sentences
s1 = "Natural language processing helps computers understand text."
s2 = "Machines can learn to interpret human language."

# Now we Process both sentences with spaCy for cleaning

CONTENT_POS = {"NOUN", "PROPN", "VERB", "ADJ", "ADV"}
tok1 = [                            # create a new list called tok1
        t.lemma_.lower()            # take each word's base form (lemma) and make it lowercase
        for t in nlp(s1)            # for every token (word) in the analyzed sentence s2
        if t.is_alpha               # keep only alphabetic words (no numbers or symbols)
        and not t.is_stop           # remove stopwords (like 'the', 'is', 'and')
        and t.pos_ in CONTENT_POS   # keep only important parts of speech (NOUN, VERB, ADJ, etc.)
        ]

tok2 = [t.lemma_.lower() for t in nlp(s2) if t.is_alpha and not t.is_stop and t.pos_ in CONTENT_POS]

# Function to average word vectors into one sentence vector
def sent_vec(model, tokens):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# Compute sentence vectors for both models
v1_w2v, v2_w2v = (                 # store two sentence vectors (for sentence 1 and sentence 2)
    sent_vec(w2v, tok1),           # create a vector for the first sentence using Word2Vec model and its tokens
    sent_vec(w2v, tok2)            # create a vector for the second sentence using Word2Vec model and its tokens
)

v1_ft,  v2_ft  = sent_vec(ft, tok1), sent_vec(ft, tok2)

# Let's Compare sentence similarity using cosine similarity
print("\n Sentence Similarity Scores:")
print("Word2Vec similarity :", float(cosine_similarity([v1_w2v], [v2_w2v])[0][0]))
print("FastText similarity :", float(cosine_similarity([v1_ft],  [v2_ft])[0][0]))



 Sentence Similarity Scores:
Word2Vec similarity : 0.875874936580658
FastText similarity : 0.993188738822937


In [ ]:
# SHOW MOST COMMON WORDS FROM THE PDF

# Flatten list of sentences to count individual words
all_words = [word for sent in sentences for word in sent]
common = Counter(all_words).most_common(10)

print("\n Most Common Words in the Paper:")
for word, freq in common:

    print(f"{word:15} → {freq} times")



 Most Common Words in the Paper:
word            → 655 times
language        → 304 times
cross           → 303 times
lingual         → 303 times
model           → 222 times
embedding       → 212 times
et              → 212 times
al              → 212 times
bilingual       → 177 times
embed           → 143 times


TASK 2

In [ ]:
#NSTALL AND IMPORT REQUIRED LIBRARIES

!pip install -q scikit-learn sentence-transformers

# sklearn → gives us evaluation metrics and cosine similarity
# sentence-transformers → lets us use a pretrained BERT model for comparison
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np


In [ ]:
# now let's try an example sentence to compare

# We'll compare sentences that mean roughly the same thing
s1 = "Natural language processing helps computers understand text."
s2 = "Machines can learn to interpret human language."

print(" Sentences ready for comparison:")
print("1.", s1)
print("2.", s2)


 Sentences ready for comparison:
1. Natural language processing helps computers understand text.
2. Machines can learn to interpret human language.


In [ ]:
# STEP 3: BASELINE METHOD — TF-IDF COSINE SIMILARITY

# TF-IDF → converts sentences into numerical vectors based on word importance
tfidf = TfidfVectorizer()
X = tfidf.fit_transform([s1, s2])  # create vectors for both sentences

# Cosine similarity → measures how similar the two sentence vectors are (0 = different, 1 = same)
tfidf_sim = cosine_similarity(X[0], X[1])[0][0]
print(f"TF-IDF Similarity: {tfidf_sim:.3f}")


TF-IDF Similarity: 0.078


In [ ]:
# STEP 4: CONTEXTUAL METHOD — BERT EMBEDDINGS

# Loaded a small pretrained BERT model (already understands English)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode both sentences into dense vectors
v1 = model.encode(s1)
v2 = model.encode(s2)

# Compute cosine similarity between the two sentence embeddings
bert_sim = cosine_similarity([v1], [v2])[0][0]
print(f"BERT Similarity: {bert_sim:.3f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

BERT Similarity: 0.637


In [ ]:
# STEP 5: COMPARE WITH MY EARLIER FASTTEXT / WORD2VEC RESULTS

# Let Suppose these were my earlier results from Task 1
w2v_sim  = 0.41   # example value
ft_sim   = 0.72   # example value

# Let's Make a small comparison table
print("\n COMPARISON TABLE")
print(f"{'Model':15} | {'Similarity'}")
print("-"*30)
print(f"{'TF-IDF':15} | {tfidf_sim:.3f}")
print(f"{'Word2Vec':15} | {w2v_sim:.3f}")
print(f"{'FastText':15} | {ft_sim:.3f}")
print(f"{'BERT':15} | {bert_sim:.3f}")


 COMPARISON TABLE
Model           | Similarity
------------------------------
TF-IDF          | 0.700
Word2Vec        | 0.410
FastText        | 0.720
BERT            | 0.300
